<a href="https://colab.research.google.com/github/mr-cri-spy/SLM/blob/main/Semantic_News_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Installing the necessary libraries
!pip install datasets
!pip install torch[cpu]
!pip install sentence-transformers

In [7]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import torch
import os

In [9]:
dataset = load_dataset("cnn_dailymail", '3.0.0', split="test")
dataset

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 11490
})

In [10]:
df = dataset.to_pandas().sample(2000, random_state=42)
df

,article,highlights,id
1516,Down Augusta way they say the azaleas are in f...,Justin Rose bounced back from Florida misery b...,58aefdc7ca85968aa11e16ea4099506cb474f759
1393,There was no special treatment for Lewis Fergu...,Lewis Ferguson fell from Merrion Square at Win...,8c2e48d24a3e2cf1be5d242f09ae34bf68ccbd6e
10560,When emergency crews received a call saying 's...,"Woman reported 'someone' had been run over, bu...",16269bfc102681f55a7fbfb6e26c7a52d982e09c
11457,A loving boyfriend has granted his girlfriend ...,"Guo Kai and girlfriend Dong Hui, 22, had plann...",18514a002a1a244a68a560c63c4471af98f72a73
647,(CNN)Sunday's announcement that Corinthian Col...,"David Wheeler: Corinthian, considered a ""preda...",9efbe27504b041e7f5e846a3c6898702c0e82427
...,...,...,...
8613,An Arizona man who suffers from an extreme cas...,"Bob Heslip, 50, suffers from Neurofibromatosis...",a83bf885303c4b3d4dae0fee56abb8b23f5d17b3
5151,A Russian military analyst has said that his c...,Konstantin Sivkov wrote that Russia needs a ne...,35992ecfcb43f6eef7454a99a2fceb9b4f975bd3
1195,Stoke boss Mark Hughes is in no doubt it is hi...,Stoke City have lost all eight trips to Chelse...,415f58a48d21bed99b9b10bc432f0966924d1a52
4604,"Top marks for Saskia, the girlfriend of an Ars...","Twitter user Saskia, 17, posted 'Arsenal exam'...",21c83ad3788158e7e06294045138d2590a7f74e5


In [13]:
model = SentenceTransformer("all-MiniLM-L6-v2")
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [16]:
passage_embeddings = list(model.encode(df['highlights'].to_list(), show_progress_bar=True))
passage_embeddings[0].shape

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

(384,)

In [17]:
query = "Find me some articles about technology and artificial intelligence"

In [20]:
query_embedding = model.encode(query)
similarities = util.cos_sim(query_embedding, passage_embeddings)

top_indices = torch.topk(similarities.flatten(), 3).indices
top_relevant_passages = [df.iloc[x.item()]['highlights'][:200] + "..." for x in top_indices]
top_relevant_passages

['Lea Coligado, 21, a Stanford computer science student, is the creator of the blog showcasing talented women in tech .\nThe profiling blog has already attracted women from big hitting tech companies lik...',
 "Robots are made out of old car parts and have CCTV cameras as heads .\nOriginally created by artist Giles Walker for a show called 'Peepshow'....",
 "Modular Robotic Vehicle developed at NASA's Johnson Space Center .\nEach wheel can be controlled independently and turn 180 degrees .\nShows off the technologies that could let man move across other pla..."]

In [24]:
def find_relevant_news(query):
    # Encode the query using the same model
    query_embedding = model.encode(query)

    # Calculate the cosine similarity between the query and passage embeddings
    similarities = util.cos_sim(query_embedding, passage_embeddings)

    # Get the indices of the top 3 most similar passages
    top_indices = torch.topk(similarities.flatten(), 3).indices

    # Retrieve the summaries of the top 3 passages and truncate them to 160 characters
    top_relevant_passages = [df.iloc[x.item()]["highlights"][:160] + "..." for x in top_indices]

    return top_relevant_passages

In [25]:
# Example queries to explore
print(find_relevant_news("Natural disasters"))
print(find_relevant_news("Law enforcement and police"))
print(find_relevant_news("Politics, diplomacy and nationalism"))


['The earthquake that struck Nepal has left thousands of Nepalis without shelter .\nTorrential rains making situation worse; food and drinking water supplies could...', "Patients flood hospitals in Nepalese capital after devastating earthquake .\nCNN's Dr. Sanjay Gupta helps with girl's operation at Nepalese medical team's reques...", "Several of Nepal's best known landmarks have been destroyed by the earthquake of April 25 .\nBut outside the capital Kathmandu there is hope that many have survi..."]
["Police officers have shut down an enormous 1000 rave in Sydney's East .\nThey were called to abandoned industrial area in Botany on Saturday night .\nPolice were ...", 'Gang in North-West allegedly beating up and blackmailing string of men .\nThey pose as children online and attack those who agree to meet them .\nMan punched in c...', 'Police in London are trying to catch the gang which staged a multi-million heist during the Easter vacation .\nFormer police commander: Such crimes require

In [26]:
def clear_screen():
    os.system("clear")

In [27]:
def interactive_search():
    print("Welcome to the Semantic News Search!\n")
    while True:
        print("Type in a topic you'd like to find articles about, and I'll do the searching! (Type 'exit' to quit)\n> ", end="")

        query = input().strip()

        if query.lower() == "exit":
            print("\nThanks for using the Semantic News Search! Have a great day!")
            break

        print("\n\tHere are 3 articles I found based on your query: \n")

        passages = find_relevant_news(query)
        for passage in passages:
            print("\n\t" + passage)

        input("\nPress Enter to continue searching...")
        clear_screen()

In [28]:
# Start the interactive search
interactive_search()

Welcome to the Semantic News Search!

Type in a topic you'd like to find articles about, and I'll do the searching! (Type 'exit' to quit)
> sports

	Here are 3 articles I found based on your query: 


	Injuries sustained in rugby can range from bruises to spinal cord damage .
Academics say government plans to increase school rugby games is risky .
Professor's ...

	Amol Gupta was playing in a match on Roosevelt Island in April 2013 when he injured himself .
Gupta suffered a broke nose and elbow, and damaged his spine when ...

	Relationship issues and sexual problems discussed over a pint with mates .
Hidden cameras recorded 70 hours of footage at Lord Nelson, east London .
Cameras did...

Press Enter to continue searching...finance
Type in a topic you'd like to find articles about, and I'll do the searching! (Type 'exit' to quit)
> finance

	Here are 3 articles I found based on your query: 


	David Cameron and Boris Johnson last night warned of a looming 'crisis'
They joined forces t